In [ ]:
import smtplib
from email.message import EmailMessage
from schedule import every, repeat, run_pending
import time
import psycopg2
import csv
from datetime import datetime
import os

# Database connection parameters
db_params = {
    'dbname': 'College automated reporting system',
    'user': 'postgres',
    'password': 'Automated@12',
    'host': 'localhost',
    'port': 5432
}

# Reconnect to the database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Function to fetch faculty emails
def get_faculty_emails():
    cursor.execute('SELECT email FROM "Clg automated"."Faculty"')
    rows = cursor.fetchall()
    return [row[0] for row in rows]

# Query attendance data and generate monthly report
def query_monthly_attendance():
    query = '''
    SELECT 
        s."Name" AS student_name, 
        sb."name" AS subject_name, 
        total_classes.total_classes AS total_classes,
        COUNT(a.id) AS present,
        (total_classes.total_classes - COUNT(a.id)) AS absent,
        (COUNT(a.id) * 100.0 / total_classes.total_classes) AS attendance_percentage
    FROM 
        "Clg automated"."Attendance" a
    JOIN 
        "Clg automated"."Student" s ON a.student_id = s.id
    JOIN 
        "Clg automated"."Subject" sb ON a.subject_id = sb.id
    JOIN 
        (
            SELECT 
                subject_id, 
                COUNT(DISTINCT date) AS total_classes
            FROM 
                "Clg automated"."Attendance"
            GROUP BY 
                subject_id
        ) AS total_classes ON a.subject_id = total_classes.subject_id
    GROUP BY 
        s."Name", 
        sb."name", 
        total_classes.total_classes
    ORDER BY 
        s."Name", 
        sb."name";
    '''
    cursor.execute(query)
    return cursor.fetchall()

def generate_csv(filename, header, data):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(data)
    return filename

def generate_monthly_report():
    data = query_monthly_attendance()
    header = ["Student Name", "Subject Name", "Total Classes", "Present", "Absent", "Attendance Percentage"]
    return generate_csv("monthly_attendance_report.csv", header, data)

def send_email(subject, body, to, attachment):
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = "support@aptpath.in"
    msg['To'] = to
    msg.set_content(body)
    
    with open(attachment, 'rb') as file:
        file_data = file.read()
        file_name = os.path.basename(attachment)
    msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
    
    with smtplib.SMTP('smtp.office365.com', 587) as smtp:
        smtp.starttls()
        smtp.login("support@aptpath.in", "btpdcnfkgjyzdndh")
        smtp.send_message(msg)

def send_monthly_report():
    monthly_report = generate_monthly_report()
    faculty_emails = get_faculty_emails()
    for email in faculty_emails:
        send_email(
            subject="Monthly Attendance Report",
            body="Please find the monthly attendance report attached.",
            to=email,
            attachment=monthly_report
        )

# Schedule the monthly report generation and sending
@repeat(every(7).days.at("17:16"))
def job_monthly():
    send_monthly_report()

def start_scheduler():
    while True:
        run_pending()
        time.sleep(1)

if __name__ == "__main__":
    # Directly generate and send the report for immediate execution
    send_monthly_report()

    # Start the scheduler for future reports
    start_scheduler()

# Close the database connection
cursor.close()
conn.close()
